In [36]:
from langchain_ollama import OllamaLLM
from langchain.prompts import PromptTemplate
from typing import TypedDict
from langgraph.graph import StateGraph, START, END

In [37]:
llm = OllamaLLM(model="gemma3:1b")

In [38]:
class Blogstate(TypedDict):
    topic: str
    outline: str
    content: str


In [39]:
graph = StateGraph(Blogstate)

In [40]:
def create_outline(state: Blogstate) -> Blogstate:
    title = state['topic']
    prompt = PromptTemplate(
        template="Create a small outline for a blog post about {title}.",
        input_variables=["title"]
    )

    response = llm.invoke(prompt.format(title=title))

    state['outline'] = response
    return state
    

In [41]:
def create_blog(state: Blogstate) -> Blogstate:
    outline = state['outline']
    prompt = PromptTemplate(
        template= "write me a small blog about the following outline: {outline}",
        input_variables=["outline"]
    )

    response = llm.invoke(prompt.format(outline=outline))

    state['content'] = response
    return state

In [42]:
graph.add_node("create_outline", create_outline)
graph.add_node("create_blog", create_blog)

In [43]:
graph.add_edge(START, "create_outline" )
graph.add_edge("create_outline", "create_blog")
graph.add_edge("create_blog", END)

In [44]:
workflow = graph.compile()

In [45]:
topic = {
    'topic': "How to use LangGraph with Ollama"
}
final_state = workflow.invoke(topic)

In [46]:
print(final_state)

{'topic': 'How to use LangGraph with Ollama', 'outline': 'Okay, here’s a small outline for a blog post about How to use LangGraph with Ollama, designed to be informative and helpful for beginners:\n\n**Blog Post Title: Unleash Your LLM Potential: A Beginner\'s Guide to LangGraph with Ollama**\n\n**I. Introduction (approx. 100-150 words)**\n\n*   Hook: Briefly explain the power of LangGraph – it’s a fantastic tool for understanding and improving LLM performance.\n*   What is LangGraph? (Keep it simple - a graph visualization of LLM behavior)\n*   Why use it with Ollama? (Highlight the benefits: easier debugging, insight, potential improvements)\n*   Briefly state the post\'s goal: “This guide will walk you through setting up and using LangGraph with Ollama to gain valuable insights into your LLM’s responses.”\n\n\n**II. What is LangGraph? (approx. 100-150 words)**\n\n*   Explain the core concept: LangGraph is a graph-based visualization tool for LLM behavior.\n*   Key Features: Briefly 